In [1]:
import json,os,sys,logging,pprint,re,string,courseraLib,random

In [16]:
sample_rate=10 #1/1000
path='/home/yewenhe0904/Developing/ds-capstone-project/testing-data/mixed-testing.txt'
testing_set=open(path)


# Initial call to print 0% progress
i = 0
l = len(list(open(path)))
l

Testset=[[], [], [], [], [], [], [], [], [], []]
Testset

courseraLib.printProgressBar(i, l, prefix = 'TestSet'+":", suffix = 'Complete')
for line in testing_set:
    p=random.randint(1,1000)
    if (p)<=sample_rate:
#         print('\n',p)
        EoSentence="[.,;!?]+"
        line_test=re.sub(EoSentence,"\n",line)
        line_test=line_test.split("\n")
        line_in=line_test[0]
        if len(line_in)<5:
            continue
        if len(line_in)>15:
            line_in=line_in[random.randint(5,10)]
        Testset[(p-1)].append(line_test[0])
    i+=1
    courseraLib.printProgressBar(i, l, prefix = 'TestSet'+":", suffix = 'Complete')

In [123]:
nGramDict=\
json.load(open\
('/home/yewenhe0904/Developing/ds-capstone-project/ignoredFiles/Sample-Dict/mixed-nGramDict_10_1.json'))

def cleanInputPredict(test_input):
    cleanedInput=[]
    test_tokens=test_input.strip().lower().split()
    test_input=re.sub('\n+'," ",test_input)
    for item in test_tokens:
        item = item.strip(string.punctuation)
        item = re.sub(r"^.*\d+.*$","<quantity>",item)
        cleanedInput.append(item)
    return cleanedInput

def predictCore(input_tokens,output_list,out_len=5):  
    input_tokens_o=input_tokens # backup original string
    if len(input_tokens)<1:
        output_list+=(nGramDict['_r'])
    else:
        # replace by word stem
        if '_word2stem' in nGramDict:
            if input_tokens[0] in nGramDict['_word2stem']['_stem']:
                input_tokens[0]=nGramDict['_word2stem']['_stem'][input_tokens[0]]
                
        # word to wid
#         print(input_tokens)
        input_idtokens=[]
        for w in input_tokens:
            w.strip()
            if w in nGramDict['_word2id']['_word']:
                input_idtokens.append(nGramDict['_word2id']['_word'][w])
            else:
                input_idtokens.append('NA')
        
        input_tokens=input_idtokens
#         print(input_tokens)
        # probe if in ngram record
        nGram_flag=True
        i=0
        p_Dict=nGramDict['_n']
        while nGram_flag:
#             print(i)
            if i==len(input_tokens):
                break
            if (input_tokens[i] in p_Dict):
                r_wids=p_Dict[input_tokens[i]]['_r']
                if ('_n' in p_Dict[input_tokens[i]]):
                    p_Dict=p_Dict[input_tokens[i]]['_n']
                    i+=1
                else:
                    if i<len(input_tokens)-1:
                        nGram_flag=False
                    break
            else:
                nGram_flag=False
                break
        
        # if true
        if nGram_flag:
            output_list+=r_wids
#             print(output_list)
#             print('find')
            if len(output_list)<out_len:
                del input_tokens_o[0]
                predictCore(input_tokens_o,output_list,out_len=out_len)
        # else false
        else:
            del input_tokens_o[0]
            predictCore(input_tokens_o,output_list,out_len=out_len)

def predictNgram(test_input,out_len):
    #clean input string
    input_tokens=cleanInputPredict(test_input)
    if len(input_tokens)>=nGramDict['_model']:
        input_tokens=input_tokens[(-nGramDict['_model']+1):]
#         print(input_tokens)
    output_list=[]
    output_wlist=[]
    output=[]
    predictCore(input_tokens,output_list,out_len)
    for wid in output_list:
        output_wlist.append(nGramDict['_id2word']['_id'][wid])
    eosflag=True
#     print(output_wlist)
    for word in output_wlist:
        if word=='<eos>':
            if eosflag:
                output.append(',')
                output.append('.')
            eosflag=False
        elif word=="<quantity>":
            continue
        else:
            output.append(word)
        #dedupe
        output_dp=[output[0]]
        if len(output)>1:
            for i in range(1,len(output)):
                if output[i] not in output[:i]:
                    output_dp.append(output[i])
        if len(output_dp)>out_len:
            output_dp=output_dp[0:out_len]
    return output_dp

In [125]:
t_input="donald trump"
predictNgram(t_input,10)

[',', '.', 'are', 'taj', 'the', 'all', 'to', 'and', 'a']

In [122]:
i=0
total_t=0
right_t=0
for t in Testset:
    i+=1
    print('Test',i,":")
    total=0
    right=0
    print('length of current test:',len(t))
    for s in t:
        stest=s.strip().lower().split()
        key=stest[-1]
        del stest[-1]
        q_input=' '.join(stest)
        if key in predictNgram(q_input,20):
            right+=1
        total+=1
    print(right,'/',total,str(round(right/total*100,2))+'%')
    total_t+=total
    right_t+=right
print('TOTAL AVERAGE: ',right_t,'/',total_t,str(round(right_t/total_t*100,2))+'%')

    

Test 1 :
length of current test: 2824
714 / 2824 25.28%
Test 2 :
length of current test: 2753
677 / 2753 24.59%
Test 3 :
length of current test: 2783
726 / 2783 26.09%
Test 4 :
length of current test: 2771
721 / 2771 26.02%
Test 5 :
length of current test: 2604
659 / 2604 25.31%
Test 6 :
length of current test: 2745
687 / 2745 25.03%
Test 7 :
length of current test: 2737
677 / 2737 24.74%
Test 8 :
length of current test: 2784
673 / 2784 24.17%
Test 9 :
length of current test: 2698
732 / 2698 27.13%
Test 10 :
length of current test: 2759
736 / 2759 26.68%
TOTAL AVERAGE:  7002 / 27458 25.5%
